In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score, roc_auc_score 
import scipy.stats as st

In [3]:
ods = pd.read_pickle('ods_scores.pkl')
# rrcf = pd.read_pickle('rrcf_scores.pkl')

In [4]:
def precision_at_k(labels, scores, k=5):
    scores = pd.Series(scores)
    labels = pd.Series(labels)
    indexes = scores.sort_values(ascending=False)[:k].index
    return labels[indexes].sum()/len(indexes)

In [5]:
def get_mean_conf_interval(df):
    
    res = {}
    index = []
    
    for key in ['AP', 'ROC', 'P@3', 'P@5']:
        
        res['mean_'+key] = df[key].mean()
        index.append('mean_'+key)
        
        res['ci_'+key] = res['mean_'+key] - st.t.interval(0.95, len(df[key].dropna())-1, loc=df[key].dropna().mean(), scale=st.sem(df[key].dropna()))[0]
        index.append('ci_'+key)
        
    return pd.Series(res, index)

In [7]:
ods['AP'] = ods.apply(lambda row: average_precision_score(row['label'], row['scores']), axis=1)
# rrcf['AP'] = rrcf.apply(lambda row: average_precision_score(row['label'], row['scores']), axis=1)

ods['ROC'] = ods.apply(lambda row: roc_auc_score(row['label'], row['scores']), axis=1)
# rrcf['ROC'] = rrcf.apply(lambda row: roc_auc_score(row['label'], row['scores']), axis=1)

ods['P@3'] = ods.apply(lambda row: precision_at_k(row['label'], row['scores'], k=3), axis=1)
# rrcf['P@3'] = rrcf.apply(lambda row: precision_at_k(row['label'], row['scores'], k=3), axis=1)

ods['P@5'] = ods.apply(lambda row: precision_at_k(row['label'], row['scores'], k=5), axis=1)
# rrcf['P@5'] = rrcf.apply(lambda row: precision_at_k(row['label'], row['scores'], k=5), axis=1)

In [8]:
ods_res = ods.groupby('dataset').apply(get_mean_conf_interval)
ods_res

/Users/andrian/opt/anaconda3/envs/ods/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1920: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
/Users/andrian/opt/anaconda3/envs/ods/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1921: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


,mean_AP,ci_AP,mean_ROC,ci_ROC,mean_P@3,ci_P@3,mean_P@5,ci_P@5
dataset,,,,,,,,
BGP_testbed_10,0.827151,0.137313,0.982870,0.017061,0.909091,0.104601,0.763636,0.168013
BGP_testbed_2,0.892658,0.058081,0.985031,0.015995,0.969697,0.067519,0.963636,0.054352
BGP_testbed_3,0.899628,0.074074,0.987264,0.014107,0.969697,0.067519,0.872727,0.090587
BGP_testbed_5,0.928133,0.061088,0.994902,0.005570,1.000000,NaN,1.000000,NaN
BGP_testbed_9,0.598972,0.119242,0.923133,0.035965,0.787879,0.150978,0.563636,0.117414


In [9]:
rrcf_res = rrcf.groupby('dataset').apply(get_mean_conf_interval)
rrcf_res

NameError: name 'rrcf' is not defined

In [10]:
tune = ['BGP_testbed_2', 'BGP_testbed_3']
test = ['BGP_testbed_5', 'BGP_testbed_9', 'BGP_testbed_10']

In [12]:
print(get_mean_conf_interval(ods[(ods['dataset'].isin(test))]).round(3))
print(get_mean_conf_interval(rrcf[(rrcf['dataset'].isin(test))]).round(3))

mean_AP     0.785
ci_AP       0.075
mean_ROC    0.967
ci_ROC      0.016
mean_P@3    0.899
ci_P@3      0.063
mean_P@5    0.776
ci_P@5      0.088
dtype: float64


NameError: name 'rrcf' is not defined